# Phase III: First ML Proof of Concept
## Solar Flares Peaks and Lengths

 - Baer Istok
 - Fiona Donohue
 - Harshini Moon
 - Evelyn Fabel

In [73]:
import requests
import pandas as pd
import numpy as np

api_key = 'f2asD6aVakycwYB4bQd6ANE3shr74umMDWTurH5x'
end_date = '2024-10-20'
start_date = '2024-09-20'


url = f'https://api.nasa.gov/DONKI/FLR?startDate={start_date}&endDate={end_date}&api_key={api_key}'

# Retrieves the data
response = requests.get(url)

# Stores the retreived data in a list
data = response.json()

In [74]:
from datetime import datetime

# Converting the list of data into a dataframe
df = pd.DataFrame(data)

# Removing the excess data
df = df.drop(columns=['instruments', 'sourceLocation', 'activeRegionNum',
                      'note', 'submissionTime', 'versionId', 'link', 'linkedEvents'], axis=1)

# Removing the numerical aspect of the class type so that it can be more categorical
# Can be X-class, M-class, or C-class
df['classType'] = df['classType'].apply(lambda x: x[:1])

# Converting the time strings into datetime objects
df['beginTime'] = df['beginTime'].apply(lambda x: datetime.fromisoformat(x))
df['peakTime'] = df['peakTime'].apply(lambda x: datetime.fromisoformat(x))
df['endTime'] = df['endTime'].apply(lambda x: datetime.fromisoformat(x))

# Determining the length of each solarflare
df['length'] = df['endTime'] - df['beginTime']

# When the peak occured over the course of the solar flare on a scale from 0 to 1
# I. e. if a peak occurs closer to the beginning of the flare it is closer to 0
df['peakPosition'] = (df['peakTime'] - df['beginTime']) / df['length']

# Cleaning the categorical data for analysis
df = pd.get_dummies(df, columns=['classType'], dtype='float')


df.head(15)

,flrID,beginTime,peakTime,endTime,length,peakPosition,classType_C,classType_M,classType_X
0,2024-09-22T21:12:00-FLR-001,2024-09-22 21:12:00+00:00,2024-09-22 21:39:00+00:00,2024-09-22 22:05:00+00:00,0 days 00:53:00,0.509434,0.0,1.0,0.0
1,2024-09-23T14:23:00-FLR-001,2024-09-23 14:23:00+00:00,2024-09-23 14:56:00+00:00,2024-09-23 15:34:00+00:00,0 days 01:11:00,0.464789,0.0,1.0,0.0
2,2024-09-25T22:57:00-FLR-001,2024-09-25 22:57:00+00:00,2024-09-25 23:12:00+00:00,2024-09-25 23:23:00+00:00,0 days 00:26:00,0.576923,0.0,1.0,0.0
3,2024-09-26T19:41:00-FLR-001,2024-09-26 19:41:00+00:00,2024-09-26 19:49:00+00:00,2024-09-26 19:54:00+00:00,0 days 00:13:00,0.615385,1.0,0.0,0.0
4,2024-09-26T22:58:00-FLR-001,2024-09-26 22:58:00+00:00,2024-09-26 23:12:00+00:00,2024-09-26 23:20:00+00:00,0 days 00:22:00,0.636364,0.0,1.0,0.0
5,2024-09-29T05:26:00-FLR-001,2024-09-29 05:26:00+00:00,2024-09-29 06:11:00+00:00,2024-09-29 06:48:00+00:00,0 days 01:22:00,0.548780,1.0,0.0,0.0
6,2024-09-29T12:49:00-FLR-001,2024-09-29 12:49:00+00:00,2024-09-29 12:53:00+00:00,2024-09-29 12:58:00+00:00,0 days 00:09:00,0.444444,0.0,1.0,0.0
7,2024-09-29T14:09:00-FLR-001,2024-09-29 14:09:00+00:00,2024-09-29 14:24:00+00:00,2024-09-29 14:31:00+00:00,0 days 00:22:00,0.681818,0.0,1.0,0.0
8,2024-09-29T14:31:00-FLR-001,2024-09-29 14:31:00+00:00,2024-09-29 14:41:00+00:00,2024-09-29 14:46:00+00:00,0 days 00:15:00,0.666667,0.0,1.0,0.0
9,2024-09-30T23:37:00-FLR-001,2024-09-30 23:37:00+00:00,2024-09-30 23:59:00+00:00,2024-10-01 00:23:00+00:00,0 days 00:46:00,0.478261,0.0,1.0,0.0


In [75]:
# keeps only the column that the machine will learn from (given length and peak position predict the type)
df['type'] = df.apply(lambda row: 'C' if row['classType_C'] == 1 else ('M' if row['classType_M'] == 1 else 'X'), axis=1)
df = df.drop(['flrID', 'beginTime', 'peakTime', 'endTime', 'classType_C', 'classType_M', 'classType_X'], axis=1)

df

,length,peakPosition,type
0,0 days 00:53:00,0.509434,M
1,0 days 01:11:00,0.464789,M
2,0 days 00:26:00,0.576923,M
3,0 days 00:13:00,0.615385,C
4,0 days 00:22:00,0.636364,M
...,...,...,...
83,0 days 00:13:00,0.615385,M
84,0 days 00:16:00,0.687500,M
85,0 days 00:25:00,0.600000,M
86,0 days 00:15:00,0.533333,M


In [78]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

df['length'] = pd.to_timedelta(df['length'])
df['length'] = df['length'].dt.total_seconds()

X = df.drop(['type'], axis=1)
y = df['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
new_data = pd.DataFrame([[3000, 0.489275], [3075, 0.500738]], columns=X.columns)

predictions = model.predict(X_test)
predictions

score = accuracy_score(y_test, predictions)
score

0.7777777777777778

In [77]:
from sklearn import tree

tree.export_graphviz(model, 
                     out_file='solar-flare-predictions.dot', 
                     feature_names=['length', 'peakPosition'],
                     class_names=sorted(y.unique()),
                     label='all',
                     rounded=True,
                     filled=True)